In [12]:
import tensorflow as tf
def run_with_location_trace(sess,op):
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    sess.run(op,options=run_options,run_metadata=run_metadata)
    for device in run_metadata.step_stats.dev_stats:
        print(device.device)
        for node in device.node_stats:
            print('  ',node.node_name)
tasks = ['localhost:100014','localhost:100015']
jobs = {'local':tasks}
cluster = tf.train.ClusterSpec(jobs)
server1 = tf.train.Server(cluster,job_name='local',task_index=0)
server2 = tf.train.Server(cluster,job_name='local',task_index=1)
tf.reset_default_graph()
sess1 = tf.Session(server1.target)
sess2 = tf.Session(server2.target)

with tf.device('/job:local/task:0'):
    var1 = tf.Variable(initial_value=0.0,name='var1')
with tf.device('/job:local/task:1'):
    var2 = tf.Variable(initial_value=1.0,name='var2')
sess1.run(tf.global_variables_initializer())
print(sess2.run(var1.assign_add(3.0)))
run_with_location_trace(sess1,var1)
run_with_location_trace(sess2,var2)
print(sess2.run(var1))
print(sess1.run(var2))
run_with_location_trace(sess2,var1)
run_with_location_trace(sess2,var2)

3.0
/job:local/replica:0/task:0/device:CPU:0
   _SOURCE
   var1
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
   var2
3.0
1.0
/job:local/replica:0/task:1/device:CPU:0
   RecvTensor
/job:local/replica:0/task:0/device:CPU:0
   _SOURCE
   var1
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
/job:local/replica:0/task:1/device:CPU:0
   _SOURCE
   var2
